# Coffee Production Forecast via Weather Data
**Goal:** Fetch and visualize current-year weather (temperature, rainfall, anomalies) for major coffee-producing regions to estimate production trends.

**Regions Covered:** Brazil, Vietnam, Colombia, India, Ethiopia, Honduras
**Weather Variables:** Rainfall, Temperature (min, max), Solar Radiation, Soil Moisture
**Data Sources:** NASA POWER API, NOAA, ERA5/ECMWF

---

In [ ]:
# Install required packages (uncomment if needed)
# !pip install pandas numpy matplotlib seaborn requests xarray netCDF4 cartopy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from datetime import date

print('Packages imported. Ready to fetch weather data.')

In [ ]:
# Define coffee-growing regions (approximate lat/lon bounding boxes)
regions = {
    'Brazil': {'lat_min': -25, 'lat_max': -5, 'lon_min': -55, 'lon_max': -35},
    'Vietnam': {'lat_min': 10, 'lat_max': 23, 'lon_min': 102, 'lon_max': 110},
    'Colombia': {'lat_min': -5, 'lat_max': 12, 'lon_min': -78, 'lon_max': -70},
    'India': {'lat_min': 8, 'lat_max': 15, 'lon_min': 75, 'lon_max': 78},
    'Ethiopia': {'lat_min': 5, 'lat_max': 15, 'lon_min': 33, 'lon_max': 48},
    'Honduras': {'lat_min': 13, 'lat_max': 16, 'lon_min': -90, 'lon_max': -86},
}
print('Regions defined:', list(regions.keys()))

In [ ]:
# Function to fetch daily weather data from NASA POWER API for a bounding box
def fetch_nasa_power(region_name, start_date, end_date):
    coords = regions[region_name]
    base_url = 'https://power.larc.nasa.gov/api/temporal/daily/point'
    # API requires single lat/lon; we take the center of region as proxy
    lat = (coords['lat_min'] + coords['lat_max'])/2
    lon = (coords['lon_min'] + coords['lon_max'])/2
    parameters = 'T2M_MAX,T2M_MIN,PRECTOT'
    url = f'{base_url}?parameters={parameters}&start={start_date}&end={end_date}&latitude={lat}&longitude={lon}&format=JSON'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()['properties']['parameter']
        df = pd.DataFrame(data)
        df.index = pd.to_datetime(df.index)
        return df
    else:
        print('Error fetching data for', region_name)
        return None

# Example: fetch_nasa_power('Brazil', '20250101', '20250925')

In [ ]:
# Compute anomalies vs historical average (example placeholder)
def compute_anomaly(df, historical_mean):
    # df and historical_mean must have same columns
    anomaly = df - historical_mean
    return anomaly

# Example usage:
# anomaly_df = compute_anomaly(current_year_df, historical_mean_df)

In [ ]:
# Visualization example: rainfall anomaly over months
def plot_rainfall_anomaly(df, region_name):
    plt.figure(figsize=(10,4))
    df['PRECTOT'].plot(kind='bar', color='teal')
    plt.title(f'Rainfall anomaly - {region_name}')
    plt.ylabel('mm/day anomaly')
    plt.xlabel('Date')
    plt.show()

# Example usage:
# plot_rainfall_anomaly(anomaly_df, 'Brazil')

## Production Risk Index
- Combine rainfall and temperature anomalies into a simple index:
  - Positive rainfall/temperature anomaly → favorable
  - Negative or extreme anomaly → risk of lower yield
- Example formula (normalized 0-1):
```python
# risk_index = (abs(rain_anomaly) + abs(temp_anomaly)) / max_threshold
```
- Export index for each region as CSV for further ML modeling.

In [ ]:
# Export risk index / weather features to CSV
# df_risk_index.to_csv('coffee_weather_risk_index.csv')
print('Export ready.')